### Third try for widgets

See: https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html

and: https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html

This notebook is a sandbox for exploring various techniques for developing widgets for design programs.

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

from Designer import SST,show

In [2]:
SECT = None   # holds the properties of one shape load from the steel tables
WIDG = {}     # holds all the value widgets, indexed by property name

def _makeFloat(key):
    w = widgets.FloatText( value=0.0, description=key+':')
    global WIDG
    WIDG[key] = w
    return w

inputs = [_makeFloat(x.strip()) for x in 'Sx,Zx,Ix,Iy,J,Cw,D,B,T,W'.split(',')]

output = widgets.Output(layout={'border': '1px solid black'})

In [3]:
@output.capture(clear_output=True)
def on_compute_clicked(b):
    global SECT
    global WIDG
    for k,w in WIDG.items():
        if hasattr(SECT,k):
            setattr(SECT,k,w.value)
    show('Dsg,*10^3,Sx,Zx,*10^6,Ix,Iy,*10^3,J,*10^9,Cw,*,D,B,T,W',object=SECT)

In [4]:
@output.capture(clear_output=True)
def on_load_clicked(b):
    global WIDG
    with output:
        Dsg = WIDG['Dsg'].value
        ##print('Dsg =',Dsg)
        global SECT
        SECT = SST.section(Dsg)
        for key,w in WIDG.items():  # propagate values to widgets
            if hasattr(SECT,key):
                w.value = getattr(SECT,key)
    ##output.clear_output()

In [5]:
WIDG['Dsg'] = widgets.Text(value='W250x80',
                           placeholder='Shape Designation',
                           description='Dsg:',
                          tooltip='A shape designation to lookup for values.')
lb = widgets.Button(description='Load Properties',
                   tooltip='Click to load properties into entries below.')
lb.on_click(on_load_clicked)
top = widgets.HBox([WIDG['Dsg'],lb])
b = widgets.Button(description='Compute!',tooltip='Click to perform calculations')
b.on_click(on_compute_clicked)
allwidg = widgets.VBox([top]+inputs+[b])
display(allwidg)
display(output)

Output(layout=Layout(border='1px solid black'))